<a href="https://colab.research.google.com/github/SP-2-Red-Data-Mining-and-AI/solid-bassoon/blob/main/SQL_Commands_From_NIBRS_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

699 lines of code as of 4/10/2024 8:19 PM

In [ ]:
import difflib as dfl
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sqlite3
from google.colab import drive
import os
import csv

In [ ]:
drive.mount('/content/drive')
db_path = '/content/drive/MyDrive/Senior Project/All Things DATA/SQLite/Python_import/NIBRS.db'
conn = sqlite3.connect(db_path)


print("SQLite database connected at:", db_path)

Mounted at /content/drive
SQLite database connected at: /content/drive/MyDrive/Senior Project/All Things DATA/SQLite/Python_import/NIBRS.db


In [ ]:
cursor = conn.cursor()

In [ ]:
conn.close()

In [ ]:
conn.row_factory = sqlite3.Row

In [ ]:
#@title Database Schema
cursor.executescript('''
PRAGMA foreign_keys = ON;
CREATE TABLE agencies (
	yearly_agency_id INTEGER,
	agency_id INTEGER PRIMARY KEY,
	data_year INTEGER,
	ori TEXT,
	legacy_ori TEXT,
	covered_by_legacy_ori TEXT,
	direct_contributor_flag TEXT,
	dormant_flag TEXT,
	dormant_year INTEGER,
	reporting_type TEXT,
	ucr_agency_name TEXT,
	ncic_agency_name TEXT,
	pub_agency_name TEXT,
	pub_agency_unit TEXT,
	agency_status TEXT,
	state_id INTEGER,
	state_name TEXT,
	state_abbr TEXT,
	state_postal_abbr TEXT,
	division_code INTEGER,
	division_name TEXT,
	region_code INTEGER,
	region_name TEXT,
	region_desc TEXT,
	agency_type_name TEXT,
	population INTEGER,
	submitting_agency_id INTEGER,
	sai TEXT,
	submitting_agency_name TEXT,
	suburban_area_flag TEXT,
	population_group_id INTEGER,
	population_group_code TEXT,
	population_group_desc TEXT,
	parent_pop_group_code INTEGER,
	parent_pop_group_desc TEXT,
	mip_flag TEXT,
	pop_sort_order INTEGER,
	summary_rape_def TEXT,
	pe_reported_flag TEXT,
	male_officer INTEGER,
	male_civilian INTEGER,
	male_total INTEGER,
	female_officer INTEGER,
	female_civilian INTEGER,
	female_total INTEGER,
	officer_rate DECIMAL,
	employee_rate DECIMAL,
	nibrs_cert_date DATE,
	nibrs_start_date DATE,
	nibrs_leoka_start_date DATE,
	nibrs_ct_start_date DATE,
	nibrs_multi_bias_start_date DATE,
	nibrs_off_eth_start_date DATE,
	covered_flag TEXT,
	county_name TEXT,
	msa_name TEXT,
	publishable_flag TEXT,
	participated TEXT,
	nibrs_participated TEXT
);
CREATE TABLE nibrs_age (
	age_id SMALLINT PRIMARY KEY,
	age_code CHARACTER(2),
	age_name TEXT
);
CREATE TABLE nibrs_arrest_type (
	arrest_type_id SMALLINT PRIMARY KEY,
	arrest_type_code CHARACTER(1),
	arrest_type_name TEXT
);
CREATE TABLE nibrs_bias_list (
	bias_id SMALLINT PRIMARY KEY,
	bias_code CHARACTER(2),
	bias_name TEXT,
	bias_desc TEXT
);
CREATE TABLE nibrs_location_type (
	location_id BIGINT PRIMARY KEY,
	location_code CHARACTER(2),
	location_name TEXT
);
CREATE TABLE nibrs_offense_type (
	offense_code TEXT PRIMARY KEY,
	offense_name TEXT,
	crime_against TEXT,
	ct_flag CHARACTER(1),
	hc_flag CHARACTER(1),
	hc_code TEXT,
	offense_category_name TEXT,
	offense_group CHARACTER(5)
);
CREATE TABLE nibrs_prop_desc_type (
	prop_desc_id SMALLINT PRIMARY KEY,
	prop_desc_name TEXT,
	prop_desc_code CHARACTER(2)
);
CREATE TABLE nibrs_victim_type (
	victim_type_id SMALLINT PRIMARY KEY,
	victim_type_code CHARACTER(1),
	victim_type_name TEXT
);
CREATE TABLE nibrs_circumstances (
	circumstances_id SMALLINT PRIMARY KEY,
	circumstances_type CHARACTER(1),
	circumstances_code SMALLINT,
	circumstances_name TEXT
);
CREATE TABLE nibrs_criminal_act (
	data_year INTEGER,
	criminal_act_id SMALLINT,
	offense_id BIGINT,
	PRIMARY KEY (criminal_act_id, offense_id),
	FOREIGN KEY (offense_id) REFERENCES nibrs_offense (offense_id)
);
CREATE TABLE nibrs_criminal_act_type (
	criminal_act_id SMALLINT PRIMARY KEY,
	criminal_act_code CHARACTER(1),
	criminal_act_name TEXT,
	criminal_act_desc TEXT
);
CREATE TABLE nibrs_ethnicity (
	ethnicity_id SMALLINT PRIMARY KEY,
	ethnicity_code CHARACTER(1),
	ethnicity_name TEXT
);
CREATE TABLE nibrs_injury (
	injury_id SMALLINT PRIMARY KEY,
	injury_code CHARACTER(1),
	injury_name TEXT
);
CREATE TABLE nibrs_prop_loss_type (
	prop_loss_id SMALLINT PRIMARY KEY,
	prop_loss_name TEXT,
	prop_loss_desc TEXT
);
CREATE TABLE nibrs_relationship (
	relationship_id SMALLINT PRIMARY KEY,
	relationship_code CHARACTER(2),
	relationship_name TEXT
);
CREATE TABLE nibrs_weapon_type (
	weapon_id SMALLINT PRIMARY KEY,
	weapon_code TEXT,
	weapon_name TEXT,
	shr_flag CHARACTER(1)
);
CREATE TABLE ref_race (
	race_id SMALLINT PRIMARY KEY,
	race_code TEXT,
	race_desc TEXT,
	sort_order SMALLINT,
	start_year SMALLINT,
	end_year SMALLINT,
	notes TEXT
);
CREATE TABLE ref_state (
	state_id SMALLINT PRIMARY KEY,
	division_id SMALLINT,
	state_name TEXT,
	state_code TEXT,
	state_abbr TEXT,
	state_postal_abbr TEXT,
	state_fips_code TEXT,
	state_pub_freq_months SMALLINT,
	change_user TEXT
);
CREATE TABLE nibrs_arrestee (
	data_year INTEGER,
	arrestee_id BIGINT PRIMARY KEY,
	incident_id BIGINT,
	arrestee_seq_num BIGINT,
	arrest_date TIMESTAMP,
	arrest_type_id SMALLINT,
	multiple_indicator CHARACTER(1),
	offense_code TEXT,
	age_id SMALLINT,
	age_num SMALLINT,
	sex_code CHARACTER(1),
	race_id SMALLINT,
	ethnicity_id SMALLINT,
	resident_code CHARACTER(1),
	under_18_disposition_code CHARACTER(1),
	clearance_ind CHARACTER(1),
	age_range_low_num SMALLINT,
	age_range_high_num SMALLINT,
	FOREIGN KEY (arrest_type_id) REFERENCES nibrs_arrest_type (arrest_type_id),
	FOREIGN KEY (age_id) REFERENCES nibrs_age (age_id),
	FOREIGN KEY (race_id) REFERENCES ref_race (race_id),
	FOREIGN KEY (ethnicity_id) REFERENCES nibrs_ethnicity (ethnicity_id),
	FOREIGN KEY (incident_id) REFERENCES nibrs_incident (incident_id)
);
CREATE TABLE nibrs_arrestee_weapon (
	data_year INTEGER,
	arrestee_id BIGINT,
	nibrs_arrestee_weapon_id BIGINT PRIMARY KEY,
	weapon_id SMALLINT,
	FOREIGN KEY (arrestee_id) REFERENCES nibrs_arrestee (arrestee_id),
	FOREIGN KEY (weapon_id) REFERENCES nibrs_weapon_type (weapon_id)
);
CREATE TABLE nibrs_bias_motivation (
	data_year INTEGER,
	bias_id SMALLINT,
	offense_id BIGINT,
	PRIMARY KEY (bias_id, offense_id),
	FOREIGN KEY (bias_id) REFERENCES nibrs_bias_list (bias_id),
	FOREIGN KEY (offense_id) REFERENCES nibrs_offense (offense_id)
);
CREATE TABLE nibrs_month (
	data_year INTEGER,
	nibrs_month_id BIGINT PRIMARY KEY,
	agency_id BIGINT,
	month_num SMALLINT,
	inc_data_year INTEGER,
	reported_status VARCHAR(10),
	report_date TIMESTAMP,
	update_flag CHARACTER(1) DEFAULT NULL,
	orig_format CHARACTER(1) DEFAULT NULL,
	data_home VARCHAR(10),
	ddocname VARCHAR(50),
	did BIGINT,
	month_pub_status INTEGER,
	FOREIGN KEY (agency_id) REFERENCES agencies (agency_id)
);
CREATE TABLE nibrs_incident (
	data_year INTEGER,
	agency_id BIGINT,
	incident_id BIGINT PRIMARY KEY,
	nibrs_month_id BIGINT,
	cargo_theft_flag CHARACTER(1),
	submission_date TIMESTAMP,
	incident_date TIMESTAMP,
	report_date_flag CHARACTER(1),
	incident_hour SMALLINT,
	incident_status VARCHAR(100),
	data_home CHARACTER(1),
	orig_format CHARACTER(1),
	did BIGINT,
	FOREIGN KEY (agency_id) REFERENCES agencies (agency_id),
	FOREIGN KEY (nibrs_month_id) REFERENCES nibrs_month (did)
);
CREATE TABLE nibrs_offender (
	data_year INTEGER,
	offender_id BIGINT PRIMARY KEY,
	incident_id BIGINT,
	offender_seq_num SMALLINT,
	age_id SMALLINT,
	age_num CHARACTER(3),
	sex_code CHARACTER(1),
	race_id SMALLINT,
	ethnicity_id SMALLINT,
	age_range_low_num SMALLINT,
	age_range_high_num SMALLINT,
	FOREIGN KEY (incident_id) REFERENCES nibrs_incident (incident_id)
);
CREATE TABLE nibrs_offense (
	data_year INTEGER,
	offense_id BIGINT PRIMARY KEY,
	incident_id BIGINT,
	offense_code VARCHAR(5),
	attempt_complete_flag CHARACTER(1),
	location_id BIGINT,
	num_premises_entered SMALLINT,
	method_entry_code CHARACTER(1),
	FOREIGN KEY (incident_id) REFERENCES nibrs_incident (incident_id),
	FOREIGN KEY (location_id) REFERENCES nibrs_location_type (location_id)
);
CREATE TABLE nibrs_property (
	data_year INTEGER,
	property_id BIGINT PRIMARY KEY,
	incident_id BIGINT,
	prop_loss_id SMALLINT,
	stolen_count SMALLINT,
	recovered_count SMALLINT,
	FOREIGN KEY (incident_id) REFERENCES nibrs_incident (incident_id),
	FOREIGN KEY (prop_loss_id) REFERENCES nibrs_prop_loss_type (prop_loss_id)
);
CREATE TABLE nibrs_property_desc (
	data_year INTEGER,
	property_id BIGINT,
	prop_desc_id SMALLINT,
	property_value BIGINT,
	date_recovered TIMESTAMP,
	nibrs_prop_desc_id BIGINT,
	PRIMARY KEY (property_id, prop_desc_id),
	FOREIGN KEY (property_id) REFERENCES nibrs_property (property_id),
	FOREIGN KEY (prop_desc_id) REFERENCES nibrs_prop_desc_type (prop_desc_id)
);
CREATE TABLE nibrs_victim (
	data_year INTEGER,
	victim_id BIGINT PRIMARY KEY,
	incident_id BIGINT,
	victim_seq_num SMALLINT,
	victim_type_id SMALLINT,
	outside_agency_id BIGINT,
	age_id SMALLINT,
	age_num VARCHAR(2),
	sex_code CHARACTER(1),
	race_id SMALLINT,
	ethnicity_id SMALLINT,
	resident_status_code CHARACTER(1),
	age_range_low_num SMALLINT,
	age_range_high_num SMALLINT,
	FOREIGN KEY (incident_id) REFERENCES nibrs_incident (incident_id),
	FOREIGN KEY (victim_type_id) REFERENCES nibrs_victim_type (victim_type_id),
	FOREIGN KEY (age_id) REFERENCES nibrs_age (age_id),
	FOREIGN KEY (race_id) REFERENCES ref_race (race_id),
	FOREIGN KEY (ethnicity_id) REFERENCES nibrs_ethnicity (ethnicity_id)
);
CREATE TABLE nibrs_victim_circumstances (
	data_year INTEGER,
	victim_id BIGINT,
	circumstances_id SMALLINT,
	PRIMARY KEY (victim_id, circumstances_id),
	FOREIGN KEY (victim_id) REFERENCES nibrs_victim (victim_id),
	FOREIGN KEY (circumstances_id) REFERENCES nibrs_circumstances (circumstances_id)
);
CREATE TABLE nibrs_victim_injury (
	data_year INTEGER,
	victim_id BIGINT,
	injury_id SMALLINT,
	PRIMARY KEY (victim_id, injury_id),
	FOREIGN KEY (victim_id) REFERENCES nibrs_victim (victim_id),
	FOREIGN KEY (injury_id) REFERENCES nibrs_injury (injury_id)
);
CREATE TABLE nibrs_victim_offender_rel (
	data_year INTEGER,
	victim_id BIGINT,
	offender_id BIGINT,
	relationship_id SMALLINT,
	nibrs_victim_offender_id BIGINT,
	PRIMARY KEY (victim_id, offender_id),
	FOREIGN KEY (victim_id) REFERENCES nibrs_victim (victim_id),
	FOREIGN KEY (offender_id) REFERENCES nibrs_offender (offender_id),
	FOREIGN KEY (relationship_id) REFERENCES nibrs_relationship (relationship_id)
);
CREATE TABLE nibrs_victim_offense (
	data_year INTEGER,
	victim_id BIGINT,
	offense_id BIGINT,
	PRIMARY KEY (victim_id, offense_id),
	FOREIGN KEY (victim_id) REFERENCES nibrs_victim (victim_id),
	FOREIGN KEY (offense_id) REFERENCES nibrs_offense (offense_id)
);
CREATE TABLE nibrs_weapon (
	data_year INTEGER,
	weapon_id SMALLINT,
	offense_id BIGINT,
	nibrs_weapon_id BIGINT,
	PRIMARY KEY (weapon_id, offense_id),
	FOREIGN KEY (weapon_id) REFERENCES nibrs_weapon_type (weapon_id),
	FOREIGN KEY (offense_id) REFERENCES nibrs_offense (offense_id)
);
''')
conn.commit()

In [ ]:
cursor.executescript('''
PRAGMA foreign_keys = OFF;

BEGIN TRANSACTION;

CREATE TABLE new_nibrs_incident (
    data_year INTEGER,
    agency_id BIGINT,
    incident_id BIGINT PRIMARY KEY,
    nibrs_month_id BIGINT,
    cargo_theft_flag CHARACTER(1),
    submission_date TIMESTAMP,
    incident_date TIMESTAMP,
    report_date_flag CHARACTER(1),
    incident_hour SMALLINT,
    incident_status VARCHAR(100),
    data_home CHARACTER(1),
    orig_format CHARACTER(1),
    did BIGINT,
    FOREIGN KEY (agency_id) REFERENCES agencies (agency_id)
    );

INSERT INTO new_nibrs_incident SELECT * FROM nibrs_incident;

DROP TABLE nibrs_incident;
ALTER TABLE new_nibrs_incident RENAME TO nibrs_incident;

PRAGMA foreign_key_check;
''')
conn.commit()
cursor.execute('PRAGMA foreign_keys = ON;')
conn.commit()

In [ ]:
with conn:
  cursor.executescript('''
  PRAGMA foreign_keys = OFF;

  BEGIN TRANSACTION;

  CREATE TABLE new_nibrs_arrestee_weapon (
      data_year INTEGER,
      arrestee_id BIGINT,
      weapon_id SMALLINT,
      PRIMARY KEY (arrestee_id, weapon_id),
      FOREIGN KEY (arrestee_id) REFERENCES nibrs_arrestee (arrestee_id),
      FOREIGN KEY (weapon_id) REFERENCES nibrs_weapon_type (weapon_id)
  );

  INSERT INTO new_nibrs_arrestee_weapon SELECT data_year, arrestee_id, weapon_id FROM nibrs_arrestee_weapon;

  DROP TABLE nibrs_arrestee_weapon;
  ALTER TABLE new_nibrs_arrestee_weapon RENAME TO nibrs_arrestee_weapon;

  PRAGMA foreign_key_check;
  ''')
  conn.commit()
  cursor.execute('PRAGMA foreign_keys = ON;')
  conn.commit()

In [ ]:
cursor.execute('DROP TABLE nibrs_month;')
conn.commit()

In [ ]:
## Define a function to check if a table exists
def table_exists(table_name):
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    return cursor.fetchone() is not None

# List of tables to check
tables_to_check = ['agencies', 'nibrs_age', 'nibrs_arrest_type', 'nibrs_bias_list', 'nibrs_location_type', 'nibrs_offense_type', 'nibrs_prop_desc_type', 'nibrs_victim_type', 'nibrs_circumstances', 'nibrs_criminal_act', 'nibrs_criminal_act_type', 'nibrs_ethnicity', 'nibrs_injury', 'nibrs_prop_loss_type', 'nibrs_relationship', 'nibrs_weapon_type', 'ref_race', 'ref_state', 'nibrs_arrestee', 'nibrs_arrestee_weapon', 'nibrs_bias_motivation', 'nibrs_month', 'nibrs_incident', 'nibrs_offender', 'nibrs_offense', 'nibrs_property', 'nibrs_property_desc', 'nibrs_victim', 'nibrs_victim_circumstances', 'nibrs_victim_injury', 'nibrs_victim_offender_rel', 'nibrs_victim_offense', 'nibrs_weapon']

# Check if each table exists
for table_name in tables_to_check:
    if table_exists(table_name):
        print(f"Table '{table_name}' exists.")
    else:
        print(f"Table '{table_name}' does not exist.")

In [ ]:
arrestee_weapon_schema = cursor.execute("PRAGMA table_info(nibrs_arrestee_weapon)")
print(arrestee_weapon_schema.fetchall())

[(0, 'data_year', 'INTEGER', 0, None, 0), (1, 'arrestee_id', 'BIGINT', 0, None, 1), (2, 'weapon_id', 'SMALLINT', 0, None, 2)]


In [ ]:
def find_pks(table_name):
  rows = cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
  tables = [row[0] for row in rows]
  for table in tables:
    if table == table_name:
      rows = cursor.execute(f"PRAGMA table_info({table})")
      print(rows.fetchall())
      pk = list(filter(lambda x : x[5]>0, rows))
      pk_name = []
      for item in pk:
        pk_name.append(item[1])
      return pk_name
  return

def column_exists(table_name, column):
  """Finds if column exists in the current table"""
  with conn:
    cursor.execute(f"SELECT * FROM {table_name}")
    names = [description[0] for description in cursor.description]  #Column names of table
    in_table = column in names
    return in_table, names  #Return Bool, List

def replace_columns(table_name, columns, curr_yr):
  """Returns a dict of column names to replace if necessary
  Iterates over column names in table and replaces name, drops column,
  or performs no action."""

  overwrite_csv = {}
  print(f"Current Table: {table_name}")
  for col in columns:
    in_table, col_names = column_exists(table_name, col) #Call to column_exists
    if in_table == False:
      i=0
      closeness_ratios = []
      for i in range(len(col_names)):
        #Gets ratio of similarity between file column and database table column name
        similar_ratio = dfl.SequenceMatcher(None, col, col_names[i]).ratio() #Uses Difflib
        closeness_ratios.append(similar_ratio)
        i+=1
      best_match = col_names[closeness_ratios.index(max(closeness_ratios))] #Highest ratio
      if max(closeness_ratios) >= 0.75:   #Determined 0.75 was best for correct match
        print(f"Best match to {col} is {best_match}")
      elif max(closeness_ratios) < 0.75:  #Less than 0.75 will add to drop
        print(f"No close match for {col}. Will drop column.")
        best_match = "DROP"
      replace_col_names = 'r'
      #replace_col_names = str(input("   r: Replace with Best/Drop, m: Manually replace, n: No changes. ")).lower()
      if replace_col_names == 'r':  #Replace with found match if r
          overwrite_csv[col] = best_match
      elif replace_col_names == 'm':  #Manually rename if m
          user_def_col = str(input("New name: "))
          overwrite_csv[col] = user_def_col
  return overwrite_csv

def replace_files(conn, directory, table_order):
  """Replaces files with unnecessary or incorrectly named columns with new files
  USE BEFORE IMPORTING FILE!!"""
  split_dir = directory.split('/')
  folder_name = split_dir[len(split_dir)-1]
  for table_name in table_order:
    if table_name == 'agencies':
      file_name = f"{table_name}.csv"
    elif table_name == 'nibrs_month':
      file_name = "NIBRS_month.csv"
    elif table_name == 'nibrs_incident':
      file_name = "NIBRS_incident.csv"
    else:
      file_name = f"{table_name.upper()}.csv"
    file_path = os.path.join(directory, file_name)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as f:
            reader = csv.reader(f)
            columns = next(reader)  # Get column names
            overwirte_csv = replace_columns(table_name, columns, folder_name) #Get dict from replace_columns
            if overwirte_csv:
              new_table = pd.read_csv(file_path)  #Pandas Dataframe to drop/rename columns in file
              for old, new in overwirte_csv.items():
                print(f"Changing {old} to {new}") #Print changes being made
                if new == "DROP":
                  new_table.drop(columns=[old], inplace=True) #Drop bad columns
                else:
                  new_table.rename(columns={old: new}, inplace=True)  #Rename columns
              print("Overwriting CSV...")
              new_table.to_csv(file_path, index=False)  #Overwrite current file with new

def traverse_state_year_file(directory):
  folder_list = []
  for yr in ['2022']:
    for folder in os.listdir(directory):
      if len(folder) <3:
        state_folder = os.path.join(directory, folder)
        if os.path.isdir(state_folder):
          #print(f"Folder: {folder}")
            for subfolder in os.listdir(state_folder):
              if yr in subfolder:
                year_folder = os.path.join(state_folder, subfolder)
                if os.path.isdir(year_folder) and subfolder not in ['AL-2022']:
                  #print(f"  Subfolder: {subfolder}")
                  folder_list.append(year_folder)
  return folder_list

def load_tables_in_order(conn, directory, table_order):
    cursor = conn.cursor()
    for table_name in table_order:
        if table_name == 'agencies':
          file_name = f"{table_name}.csv"
        elif table_name == 'nibrs_incident':
          file_name = "NIBRS_incident.csv"
        else:
          file_name = f"{table_name.upper()}.csv"
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as f:
                reader = csv.reader(f)
                columns = next(reader)  # Get column names
                placeholders = ','.join(['?'] * len(columns))
                # Construct the SQL query with proper table name and placeholders
                insert_query = f"INSERT OR IGNORE INTO \"{table_name}\" ({','.join(columns)}) VALUES ({placeholders})"
                print("SQL query:", insert_query)  # Print SQL query for debugging
                # Iterate over rows and execute the query for each row
                for row in reader:
                    #print("Row data:", row)  # Print row data for debugging
                    cursor.execute(insert_query, row)
    conn.commit()

In [ ]:
find_pks('new_nibrs_arrestee_weapon')

In [ ]:
table_order = [
    'nibrs_age',
    'nibrs_arrest_type',
    'nibrs_bias_list',
    'nibrs_location_type',
    'nibrs_offense_type',
    'nibrs_prop_desc_type',
    'nibrs_victim_type',
    'nibrs_circumstances',
    'nibrs_criminal_act_type',
    'nibrs_ethnicity',
    'nibrs_injury',
    'nibrs_prop_loss_type',
    'nibrs_relationship',
    'nibrs_weapon_type',
    'ref_race',
    'ref_state',
    'agencies',
    #'nibrs_month', REMOVED
    'nibrs_incident',
    'nibrs_offender',
    'nibrs_offense',
    'nibrs_property',
    'nibrs_property_desc',
    'nibrs_victim',
    'nibrs_victim_circumstances',
    'nibrs_victim_injury',
    'nibrs_victim_offender_rel',
    'nibrs_victim_offense',
    'nibrs_weapon'
]

data_import_order = [
    #'agencies',
    #'nibrs_incident',
    #'nibrs_offense',
    #'nibrs_offender',
    #'nibrs_criminal_act',
    #'nibrs_arrestee',
    #'nibrs_victim',
    #'nibrs_victim_circumstances',
    #'nibrs_victim_offense',
    #'nibrs_victim_offender_rel',
    #'nibrs_victim_injury',
    #'nibrs_weapon',
    'nibrs_arrestee_weapon',
    #'nibrs_property',
    #'nibrs_property_desc'
]
#Agencies
#Month
#Incident
#Offense
#Offender
#Criminal act
#Arrestee
#Victim
#Victim Circumstances
#Victim Offense
#Victim Offender Rel
#Weapon
#Arrestee Weapon
#Property
#Property Desc


# Path to AK directory
ak_2021_directory = '/content/drive/MyDrive/Senior Project/All Things DATA/Datasets/AK/AK-2021'
ak_2022_directory = '/content/drive/MyDrive/Senior Project/All Things DATA/Datasets/AK/AK-2022'

#Path to Datasets
state_directory = '/content/drive/MyDrive/Senior Project/All Things DATA/Datasets'

directories = traverse_state_year_file(state_directory)
print(directories)
#new_directories = directories[directories.index('/content/drive/MyDrive/Senior Project/All Things DATA/Datasets/CO/CO-2021')+1:]

for dir in directories:
  split_dir = dir.split('/')
  folder_name = split_dir[len(split_dir)-1]
  print(f"Current: {folder_name}")

  #First check for inconsistent column names in file
  replace_files(conn, dir, data_import_order)

  # Load tables in order from AK-2021 directory
  load_tables_in_order(conn, dir, data_import_order)

In [ ]:
table_order = [
    'nibrs_age',
    'nibrs_arrest_type',
    'nibrs_bias_list',
    'nibrs_location_type',
    'nibrs_offense_type',
    'nibrs_prop_desc_type',
    'nibrs_victim_type',
    'nibrs_circumstances',
    'nibrs_criminal_act_type',
    'nibrs_ethnicity',
    'nibrs_injury',
    'nibrs_prop_loss_type',
    'nibrs_relationship',
    'nibrs_weapon_type',
    'ref_race',
    'ref_state',
    'agencies',
    #'nibrs_month',
    'nibrs_incident',
    'nibrs_offender',
    'nibrs_offense',
    'nibrs_property',
    'nibrs_property_desc',
    'nibrs_victim',
    'nibrs_victim_circumstances',
    'nibrs_victim_injury',
    'nibrs_victim_offender_rel',
    'nibrs_victim_offense',
    'nibrs_weapon'
]
with conn:
  length = []
  for table_name in table_order:
    print(f"{table_name}")
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    res = cursor.fetchall()
    length.append(res)
print(length)